In [ ]:
%pip install --upgrade git+https://github.com/terrierteam/pyterrier_dr.git
import pyterrier as pt
pt.utils.set_tqdm('notebook')
import pyterrier_dr

Load your index and the relevant model

If you dont have an existing TCT index, one can be created for MSMARCO as follows:
```python
index = pyterrier_dr.FlexIndex('myindex.flex')
idx_pipeline = model >> index
idx_pipeline.index(pt.get_dataset('irds:msmarco-passage').get_corpus_iter())
```

In [5]:
index_loc = '/nfs/global_indices/msmarco-passage.tct-hnp.flex'
model = pyterrier_dr.TctColBert('castorini/tct_colbert-v2-hnp-msmarco')
index = pyterrier_dr.FlexIndex(index_loc)

tokenizer_config.json:   0%|          | 0.00/334 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# standard dense retrieval
baseline = model >> index

In [8]:
# PRF pipelines
vector_prf = model >> index % 3 >> index.vec_loader() >> pyterrier_dr.VectorPrf() >> index
average_prf = model >> index % 3 >> index.vec_loader() >> pyterrier_dr.AveragePrf() >> index

In [16]:

import pyterrier as pt
from pyterrier.measures import *
pt.Experiment(
    [baseline, vector_prf, average_prf],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    [nDCG@10, AP(rel=2)@100],
    names=["TCT", "TCT >> VectorPRF", "TCT >> AveragePRF"],
    baseline=0
    )

Java started (triggered by _read_topics_singleline) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
/opt/miniconda3/lib/python3.10/site-packages/pyterrier_dr/flex/core.py:90: UserWarning: performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster
  warn("performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster")


NumpyRetriever scoring:   0%|          | 0/2159 [00:00<?, ?docbatch/s]

/opt/miniconda3/lib/python3.10/site-packages/pyterrier_dr/flex/core.py:90: UserWarning: performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster
  warn("performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster")


NumpyRetriever scoring:   0%|          | 0/2159 [00:00<?, ?docbatch/s]

/opt/miniconda3/lib/python3.10/site-packages/pyterrier_dr/flex/core.py:90: UserWarning: performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster
  warn("performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster")


...

/opt/miniconda3/lib/python3.10/site-packages/pyterrier_dr/flex/core.py:90: UserWarning: performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster
  warn("performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster")


NumpyRetriever scoring:   0%|          | 0/2159 [00:00<?, ?docbatch/s]

/opt/miniconda3/lib/python3.10/site-packages/pyterrier_dr/flex/core.py:90: UserWarning: performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster
  warn("performing exhaustive search with FlexIndex.np_retriever -- note that other FlexIndex retrievers may be faster")


NumpyRetriever scoring:   0%|          | 0/2159 [00:00<?, ?docbatch/s]

,name,nDCG@10,AP(rel=2)@100,nDCG@10 +,nDCG@10 -,nDCG@10 p-value,AP(rel=2)@100 +,AP(rel=2)@100 -,AP(rel=2)@100 p-value
0,TCT,0.720577,0.402403,NaN,NaN,NaN,NaN,NaN,NaN
1,TCT >> VectorPRF,0.732201,0.425013,24.0,11.0,0.203288,30.0,12.0,0.000902
2,TCT >> AveragePRF,0.729361,0.441232,25.0,14.0,0.505680,28.0,14.0,0.032259
